## Full info classification

### Preparation of the sets

In [ ]:
mean_sem = np.array([poem.mean(axis=0) for poem in sem_ds])
print(mean_sem.shape)
full_ds = np.hstack([style_ds, mean_sem])
print(full_ds.shape)

with open(os.path.join(path_data, "full_save.npy"), 'wb') as f:
    np.save(f, full_ds)


mean_fake_sem = np.array([poem.mean(axis=0) for poem in fake_sem_ds])
full_fake_ds = np.hstack([style_fake_ds, mean_fake_sem])
print(full_fake_ds.shape)

with open(os.path.join(path_data, "full_fake_save.npy"), 'wb') as f:
    np.save(f, full_fake_ds)

In [ ]:
with open(os.path.join(path_data, "full_save.npy"), 'rb') as f:
    full_ds = np.load(f)

with open(os.path.join(path_data, "full_fake_save.npy"), 'rb') as f:
    full_ds_fake = np.load(f)

In [ ]:
#keep the same test/Train than before
full_ds_train=np.delete(full_ds,list_test,0)
full_ds_test=np.delete(full_ds,list_train,0)

print(full_ds_train.shape)
print(full_ds_test.shape)

full_ds_mix = []
for i in range(10):
    full_ds_mix.append(full_ds_train[25*i:25*(i+1), :])
    full_ds_mix.append(full_ds_fake[10*i:10*(i+1), :])

full_ds_mix = np.vstack(full_ds_mix)
print(full_ds_mix.shape)

In [ ]:
pca_full = PCA(n_components = 50, random_state = 6)
train_pca = pca_full.fit_transform(full_ds_train)
tsne_full = TSNE(n_components = 2, perplexity = 10, learning_rate="auto", init="pca", random_state=6)
train_tsne = tsne_full.fit_transform(train_pca)

In [ ]:
test_pca = pca_full.fit_transform(full_ds_test)
test_tsne = tsne_full.fit_transform(test_pca)
fake_pca = pca_full.fit_transform(full_ds_fake)
fake_tsne = tsne_full.fit_transform(fake_pca)

In [ ]:
nb_poems_train = 250
nb_poems_test= 50
nb_poems_fake =100

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.scatter(train_tsne[:,0],train_tsne[:,1],c=color_train, s=10, cmap="rainbow", label="Train set")
plt.scatter(test_tsne[:,0],test_tsne[:,1], c=color_test,s=50, marker="^",cmap="rainbow", label="Test set")
plt.scatter(fake_tsne[:,0],fake_tsne[:,1], c=color_fake,s=100, marker="+",cmap="rainbow",label="Fake set")
plt.title("Projection with t-SNE of the full data set after standardisation")
plt.legend()

In [ ]:

scaler_full = preprocessing.StandardScaler().fit(full_ds_train)
full_ds_train = scaler_full.transform(full_ds_train)
full_ds_test = scaler_full.transform(full_ds_test)
full_ds_fake = scaler_full.transform(full_ds_fake)

In [ ]:
fig, ax = plt.subplots(6, 1, figsize=(10,5))
ax[0].boxplot(full_ds_train[:,:])
ax[1].boxplot(full_ds_fake[:,:])
ax[2].boxplot(full_ds_test[:,:])
ax[3].boxplot(full_ds_fake[:,:])
ax[4].boxplot(full_ds_train[:,:])
ax[5].boxplot(full_ds_test[:,:]);

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10,5))
ax[0].boxplot(full_ds_fake[:,0:100])
ax[1].boxplot(full_ds_train[:,0:100])
ax[2].boxplot(full_ds_test[:,0:100]);

In [ ]:
mean_full=np.mean(full_ds_train,0)  #(de chauque compo)
std_full=np.std(full_ds_train,0)


print(mean_full)
print(std_full[0:100])

In [ ]:
mean_full=np.mean(full_ds_fake,0)  #(de chauque compo)
std_full=np.std(full_ds_fake,0)

print(mean_full)
print(std_full)

In [ ]:
#dist_full = "euclidean"
#dist_full = "cityblock"

def adapt_dist(vec1,vec2):
    n = len(vec1)
    part_feat=spatial.distance.cityblock(vec1[:83],vec2[:83])
    part_emb=spatial.distance.euclidean(vec1[84:1023],vec2[84:1023])
    return part_feat+part_emb

#dist_full = adapt_dist
dist_full ="adapt_non_scale"

In [ ]:
d_train_full = spatial.distance.pdist(full_ds_train, dist_full)
distance_train_full = sr.Distance(d_train_full)
matrix_distance_train_full = distance_train_full.square_form()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.xticks(range(0, int(np.max(matrix_distance_fake_full)), 1))
plt.xlabel("Pairwise distance")
plt.ylabel("Absolute freq.")
plt.title("Full distance histogram (all authors)")
plt.hist(matrix_distance_train_full)
plt.show()

In [ ]:
#histogram by author 
fig, ax = plt.subplots(2, 5, figsize=(20,5), sharey=True)

for i, txt in enumerate(list_author): 
    ax[i//5, i%5].hist(matrix_distance_train_full[25*i:25*(i+1), :])
    ax[i//5, i%5].set_title(txt)

fig.suptitle("Full distance histogram per author ", fontsize=15)
fig.tight_layout()
fig.subplots_adjust(top=0.82)
plt.show()

In [ ]:
distributions_full = {
    "euclidean":{
        "0_38": sr.get_distribution(name="uniform", interval=[0,38]),
        "0_48": sr.get_distribution(name="uniform", interval=[0,48]),
        "0_57": sr.get_distribution(name="uniform", interval=[0,57]),
        # if you consider closest points as the same --> Ring neightbourhood
        "38_57": sr.get_distribution(name="uniform", interval=[38,57]),
        "48_105": sr.get_distribution(name="uniform", interval=[48,105]),
        },
    "cityblock":{
        "0_1000": sr.get_distribution(name="uniform", interval=[0,1000]),
        "0_1300": sr.get_distribution(name="uniform", interval=[0,1300]),
        "0_1550": sr.get_distribution(name="uniform", interval=[0,1550]),
        # if you consider closest points as the same --> Ring neightbourhood
        "1300_2500": sr.get_distribution(name="uniform", interval=[1300,2500]),
        "1000_1800": sr.get_distribution(name="uniform", interval=[1000,1800]),
        },
    "adapt_dist":{
        "0_105": sr.get_distribution(name="uniform", interval=[0,105]),
        "0_126": sr.get_distribution(name="uniform", interval=[0,126]),
        "0_150": sr.get_distribution(name="uniform", interval=[0,150]),
        # if you consider closest points as the same --> Ring neightbourhood
        "105_126": sr.get_distribution(name="uniform", interval=[105,126]),
        "105_210": sr.get_distribution(name="uniform", interval=[105,210]),
        },
    "adapt_non_scale":{
        "0_7.7": sr.get_distribution(name="uniform", interval=[0,7.7]),
        "0_9.1": sr.get_distribution(name="uniform", interval=[0,9.1]),
        "0_10.8": sr.get_distribution(name="uniform", interval=[0,10.8]),
        # if you consider closest points as the same --> Ring neightbourhood
        "6_10.8": sr.get_distribution(name="uniform", interval=[6,10.8]),
        "7.7_15": sr.get_distribution(name="uniform", interval=[7.7,15]),
        }
    
    }

In [ ]:
d_test_full = spatial.distance.pdist(full_ds_test, dist_full)
distance_test_full = sr.Distance(d_test_full)
matrix_distance_test_full = distance_test_full.square_form()

d_fake_full = spatial.distance.pdist(full_ds_fake, dist_full)
distance_fake_full = sr.Distance(d_fake_full)
matrix_distance_fake_full = distance_fake_full.square_form()


d_mix_full = spatial.distance.pdist(full_ds_mix, dist_full)
distance_mix_full = sr.Distance(d_mix_full)
matrix_distance_mix_full = distance_mix_full.square_form()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.xticks(range(0, int(np.max(matrix_distance_fake_full)), 5))
plt.xlabel("Pairwise distance")
plt.ylabel("Absolute freq.")
plt.title("Full distance histogram (all authors)")
plt.hist(matrix_distance_fake_full)
plt.show()

### Creation of the stable ranks

In [ ]:
# Hyperparams
number_instances = 300
sample_size = 30
  

links = ["single", "complete", "average", "ward"]

# dict{defaultdict{dict{dict{list}}}}
logs_full = {
    "train": defaultdict(lambda: {}),
    "mix": defaultdict(lambda: {}),
    "test": defaultdict(lambda: {}),
    "fake": defaultdict(lambda: {})

}

matrix_distance_all_full = {
    "train": matrix_distance_train_full,
    "mix": matrix_distance_mix_full,
    "test": matrix_distance_test_full,
    "fake": matrix_distance_fake_full
    
}

distance_obj_all_full = {
    "train": distance_train_full,
    "mix": distance_mix_full,
    "test": distance_test_full,
    "fake": distance_fake_full
    
}

for dataset, matrix_distance in matrix_distance_all_full.items():
    h1_computed = False
    for link in links:
        h0_sr = {}
        h1_sr = {}
        bc = {}
        for k in  distributions_full[dist_full].keys():
            h0_sr[k] = []
            h1_sr[k] = []
            bc[k] = []
            for poem in matrix_distance:
                # Sampling in interval
                p = distributions_full[dist_full][k](poem)
                s = sr.get_sample(number_instances, sample_size, p)
                
                # H0 stable rank, all samples
                f = distance_obj_all_full[dataset].get_h0sr(sample=s, clustering_method=link)
                
                # Bar code, one sample
                s_1 = sr.get_sample(1, sample_size, p)
                b_1 = distance_obj_all_full[dataset].get_bc(sample=s_1, maxdim=1, reduced=False)
                
                if not h1_computed:
                    # H1 bar code and stable rank, all samples 
                    b = distance_obj_all_full[dataset].get_bc(sample=s, maxdim=1)
                    g = sr.bc_to_sr(b, degree="H1")
                    h1_sr[k].append(g)

                h0_sr[k].append(f)
                bc[k].append(b_1)
        
        logs_full[dataset][link]["h0"] = h0_sr
        logs_full[dataset][link]["h1"] = h1_sr if not h1_computed else logs_full[dataset][links[0]]["h1"]
        h1_computed=True
        logs_full[dataset][link]["bc"] = bc

with open(os.path.join(path_data, f"logs_full_{dist_full}.p"), "wb") as f:
    pickle.dump(logs_full,  f)

In [ ]:
with open(os.path.join(path_data,f"logs_full_{dist_full}.p"), "rb" ) as f:
  logs_full =  pickle.load(f)

In [ ]:
def pipeline_full(logs, mode="all_plots", nb_poems=nb_poems, data_name="train"):
    list_integers = range(0,10) #the range and variance of this list will vary 
    cmap = cm.rainbow(np.array(list_integers)/(2*np.mean(list_integers)))
    custom_lines = [Line2D([0], [0], color=cmap[i], lw=2) for i in range(10)]
    num_poems_x_auth = nb_poems//10
    
    plotted = False
    num_splits = len(distributions_full[dist_full].keys())
    
    if mode.lower() == "all_auth" or mode.lower() == "all_plots":
        #h0 & h1 plot all authors
        links = logs.keys()
        fig, ax = plt.subplots(len(links) + 1, num_splits, figsize=(5*num_splits, 5*(len(links) + 1)))
        for link_idx, link in enumerate(links):
            h0_sr = logs[link]["h0"] 
            h1_sr = logs[link]["h1"] 
            
            for idx, key in enumerate(distributions_full[dist_full].keys()):
                i = 0
                color = -1
                for h0, h1 in zip(h0_sr[key], h1_sr[key]):
                    if i % num_poems_x_auth == 0:
                        color += 1
                        
                    h0.plot(color=cmap[color], ax=ax[link_idx, idx])
                    
                    if link_idx == 0:
                        h1.plot(color=cmap[color], ax=ax[-1, idx])

                    i += 1
                    
                if link_idx == 0:
                    ax[0, idx].set_title(f"Split {key}")
            
            ax[link_idx, 0].set_ylabel(r"$H_0$ mean $\widehat{rank}$" + f" ({link})")
            
        ax[-1, 0].set_ylabel(r"$H_1$ mean $\widehat{rank}$")

        fig.legend(custom_lines, [s for s in list_author],
                    ncol=2)

        fig.suptitle("Full stable rank per split", fontsize=20, y=0.96)
        fig.tight_layout()
        fig.subplots_adjust(top=0.92)

        # Get the bounding boxes of the axes including text decorations
        r = fig.canvas.get_renderer()
        get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())
        bboxes = np.array(list(map(get_bbox, ax.flat)), mtrans.Bbox).reshape(ax.shape)

        #Get the minimum and maximum extent, get the coordinate half-way between those
        ymax = np.array(list(map(lambda b: b.y1, bboxes.flat))).reshape(ax.shape).max(axis=1)
        ymin = np.array(list(map(lambda b: b.y0, bboxes.flat))).reshape(ax.shape).min(axis=1)
        ys = np.c_[ymax[1:], ymin[:-1]].mean(axis=1)

        # Draw a horizontal lines at those coordinates
        y = ys[-1]
        line = plt.Line2D([0,1],[y,y], transform=fig.transFigure, color="grey")
        fig.add_artist(line)
        plt.savefig(os.path.join(path_out, f"Figures/Full/{dist_full.capitalize()}/full_{data_name}_sr_all_{dist_full}.png"), dpi=100)
        plt.show()
        plotted = True
    
    if mode.lower() == "all_means" or mode.lower() == "all_plots":
        #h0 & h1 plot all authors
        links = logs.keys()
        fig, ax = plt.subplots(len(links) + 1, num_splits, figsize=(5*num_splits, 5*(len(links) + 1)))
        for link_idx, link in enumerate(links):
            h0_sr = logs[link]["h0"] 
            h1_sr = logs[link]["h1"] 
            for idx, key in enumerate(distributions_full[dist_full].keys()):
                auth_logs_h0 =  [[] for _ in range(10)]
                auth_logs_h1 =  [[] for _ in range(10)]
                i = 0
                color = -1
                for h0, h1 in zip(h0_sr[key], h1_sr[key]):
                    if i % num_poems_x_auth == 0:
                        color += 1
                    
                    auth_logs_h0[color].append(h0)
                    auth_logs_h1[color].append(h1)
                    
                    i += 1
                
                auth_idx = 0
                for h0, h1 in zip(auth_logs_h0,auth_logs_h1):
                    h0_mean = sum(h0)/len(h0)
                    h1_mean = sum(h1)/len(h1)
                    
                    h0_mean.plot(color=cmap[auth_idx], ax=ax[link_idx, idx])
                    if link_idx == 0:
                        h1_mean.plot(color=cmap[auth_idx], ax=ax[-1, idx])

                    auth_idx += 1
                
                if link_idx == 0:
                    ax[0, idx].set_title(f"Split {key}")
                
            ax[link_idx, 0].set_ylabel(r"$H_0$ mean $\widehat{rank}$" + f" ({link})")


        ax[-1, 0].set_ylabel(r"$H_1$ mean $\widehat{rank}$")

        fig.legend(custom_lines, [s for s in list_author],
                    ncol=2)

        fig.suptitle(f"Full mean stable rank per split", fontsize=20, y=0.96)
        fig.tight_layout()
        fig.subplots_adjust(top=0.92)

        # Get the bounding boxes of the axes including text decorations
        r = fig.canvas.get_renderer()
        get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())
        bboxes = np.array(list(map(get_bbox, ax.flat)), mtrans.Bbox).reshape(ax.shape)

        #Get the minimum and maximum extent, get the coordinate half-way between those
        ymax = np.array(list(map(lambda b: b.y1, bboxes.flat))).reshape(ax.shape).max(axis=1)
        ymin = np.array(list(map(lambda b: b.y0, bboxes.flat))).reshape(ax.shape).min(axis=1)
        ys = np.c_[ymax[1:], ymin[:-1]].mean(axis=1)

        # Draw a horizontal lines at those coordinates
        y = ys[-1]
        line = plt.Line2D([0,1],[y,y], transform=fig.transFigure, color="grey")
        fig.add_artist(line)
        plt.savefig(os.path.join(path_out, f"Figures/Full/{dist_full.capitalize()}/full_{data_name}_sr_mean_{dist_full}.png"), dpi=100)
        plt.show()
        plotted = True
    
    if mode.lower() == "each_auth" or mode.lower() == "all_plots":
        #h0 & h1 plot per author 
        cmap = cm.rainbow(np.array(list_integers)/(2*np.mean(list_integers)))
        color = -1
        for auth_idx, auth in enumerate(list_author):
            color += 1
            fig, ax = plt.subplots(len(links) + 1, num_splits, figsize=(5*num_splits, 5*(len(links) + 1)))
            for link_idx, link in enumerate(links):
                h0_sr = logs[link]["h0"] 
                h1_sr = logs[link]["h1"] 
                for idx, key in enumerate(distributions_full[dist_full].keys()):
                    h_0_auth = []
                    h_1_auth = []
                    i = 0
                    for h0, h1 in zip(h0_sr[key], h1_sr[key]):
                        if i//num_poems_x_auth == auth_idx:
                            h_0_auth.append(h0)
                            h_1_auth.append(h1)
                        else:
                            h0.plot(color="grey", ax=ax[link_idx, idx], alpha=0.6)
                            if link_idx == 0:
                                h1.plot(color="grey", ax=ax[-1, idx], alpha=0.6)

                        i += 1
                    
                    for h0, h1 in zip(h_0_auth, h_1_auth): 
                        h0.plot(color=cmap[color], ax=ax[link_idx, idx],)
                        if link_idx == 0:
                            h1.plot(color=cmap[color], ax=ax[-1, idx])
                            
                    if link_idx == 0:
                        ax[0, idx].set_title(f"Split {key}")
                        
                ax[link_idx, 0].set_ylabel(r"$H_0$ mean $\widehat{rank}$" + f" ({link})")


            ax[-1, 0].set_ylabel(r"$H_1$ mean $\widehat{rank}$")

            fig.suptitle(f"Full stable rank per split: {auth}", fontsize=20)
            fig.tight_layout()
            fig.subplots_adjust(top=0.92)

            # Get the bounding boxes of the axes including text decorations
            r = fig.canvas.get_renderer()
            get_bbox = lambda ax: ax.get_tightbbox(r).transformed(fig.transFigure.inverted())
            bboxes = np.array(list(map(get_bbox, ax.flat)), mtrans.Bbox).reshape(ax.shape)

            #Get the minimum and maximum extent, get the coordinate half-way between those
            ymax = np.array(list(map(lambda b: b.y1, bboxes.flat))).reshape(ax.shape).max(axis=1)
            ymin = np.array(list(map(lambda b: b.y0, bboxes.flat))).reshape(ax.shape).min(axis=1)
            ys = np.c_[ymax[1:], ymin[:-1]].mean(axis=1)

            # Draw a horizontal lines at those coordinates
            y = ys[-1]
            line = plt.Line2D([0,1],[y,y], transform=fig.transFigure, color="grey")
            fig.add_artist(line)
            auth_name = auth.replace(" ", "_").replace(".", "")
            plt.savefig(os.path.join(path_out,
                        f"Figures/Full/{dist_full.capitalize()}/full_{data_name}_sr_{auth_name}_{dist_full}.png"),
                        dpi=100)
            plt.show()
            plotted = True
            
    if not plotted:
        print(f"{mode} is not and option!")
        return -1

    return 0

In [ ]:
pipeline_full(logs_full["train"], nb_poems=nb_poems);

In [ ]:
pipeline_full(logs_full["fake"], nb_poems=nb_poems_fake, data_name="fake");

In [ ]:
links = ["single", "complete", "average", "ward"]

normal_train_res_full_mean = pd.DataFrame(columns=distributions_full[dist_full].keys())
normal_train_res_full_std = pd.DataFrame(columns=distributions_full[dist_full].keys())
mix_train_res_full_mean = pd.DataFrame(columns=distributions_full[dist_full].keys())
mix_train_res_full_std = pd.DataFrame(columns=distributions_full[dist_full].keys())
res_full = {
    "train": (normal_train_res_full_mean, normal_train_res_full_std),
    "mix": (mix_train_res_full_mean, mix_train_res_full_std)
    }

targets_full = {
    "train": np.array([i for i in range(10) for _ in range(nb_poems//10)]),
    "mix": np.array([i for i in range(10) for _ in range(nb_poems_mix//10)]),
    "test": np.array([i for i in range(10) for _ in range(nb_poems_test//10)])
    
}

num_iter = 15

for dataset, results in res_full.items():
    
    h0_kernel_train = defaultdict(lambda: {})
    h0_kernel_test = defaultdict(lambda: {})

    h1_kernel_train = {}
    h1_kernel_test = {}

    for split in distributions_full[dist_full].keys():
        for cm in links:
            train_h0_sr = logs_full[dataset][cm]["h0"][split]
            test_h0_sr = logs_full["test"][cm]["h0"][split]
            
            h0_kernel_train[split][cm] = np.array([[f.dot(g) for g in train_h0_sr] for f in train_h0_sr])
            h0_kernel_test[split][cm] = np.array([[f.dot(g) for g in train_h0_sr] for f in test_h0_sr])
        
        train_h1_sr = logs_full[dataset]["single"]["h1"][split]
        test_h1_sr = logs_full["test"]["single"]["h1"][split]
        
        h1_kernel_train[split] = np.array([[f.dot(g) for g in train_h1_sr] for f in train_h1_sr])
        h1_kernel_test[split] = np.array([[f.dot(g) for g in train_h1_sr] for f in test_h1_sr])

    acc_logs = defaultdict(lambda: [[] for _ in range(num_iter)])
    for i in range(num_iter):
        for split in distributions_full[dist_full].keys():
            acc_h0 = []
            clf_h0 = []
            
            # H1 classifier
            clf_h1 =  SVC(kernel='precomputed', probability=True)
            clf_h1.fit(h1_kernel_train[split], targets_full[dataset])
            prob_h1 = clf_h1.predict_proba(h1_kernel_test[split])
            prediction = np.argmax(prob_h1, axis=-1)
            acc_logs["h1"][i].append(accuracy_score(targets_full["test"], prediction))
            
            probs = None
            for cm in links:
                # H0 classifier
                clf = SVC(kernel='precomputed', probability=True)
                clf.fit(h0_kernel_train[split][cm], targets_full[dataset])
                clf_h0.append(clf)
                prob_h0 = clf.predict_proba(h0_kernel_test[split][cm])
                
                if probs is None:
                    probs = np.zeros_like(prob_h0)
                    
                probs += prob_h0 # (?)
                prediction = np.argmax(prob_h0, axis=-1)
                acc_logs[f"h0_{cm}"][i].append(accuracy_score(targets_full["test"], prediction))
                
                # Pairwise ensemble
                avg_prob = (prob_h1 + prob_h0)/2.0
                prediction = np.argmax(avg_prob, axis=-1)
                acc_logs[f"ensb_{cm}"][i].append(accuracy_score(targets_full["test"], prediction))
            
            # Full ensemble
            avg_prob = (prob_h1 + probs)/(len(links) + 1)
            prediction = np.argmax(avg_prob, axis=-1)
            acc_logs[f"full_ensb"][i].append(accuracy_score(targets_full["test"], prediction))

    
    for name, acc in acc_logs.items():
        results[0].loc[len(results[0].index)] = np.mean(acc, axis=0)
        results[1].loc[len(results[1].index)] = np.std(acc, axis=0)
        aux = list(results[0].index)
        aux[-1] = name
        results[0].index = aux
        results[1].index = aux
    
    results[0].to_csv(os.path.join(path_out, f"res_full_{dist_full}_{dataset}_mean_{num_iter}.csv"), index=True)
    results[1].to_csv(os.path.join(path_out, f"res_full_{dist_full}_{dataset}_std_{num_iter}.csv"), index=True)

In [ ]:
merge_mean_std(res_full["train"][0], res_full["train"][1])

In [ ]:
merge_mean_std(res_full["mix"][0], res_full["mix"][1])